# Vector boson fusion in IDM

The final notebook is [Significance_countours.ipynb](./Significance_countours.ipynb) which is based on [Sensitivity_fit.ipynb](Sensitivity_fit.ipynb) which is based on [Sensitivity_plots.ipynb](Sensitivity_plots.ipynb) by JD

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Install root 5

### Prerequisites 
```bash
apt-get install cmake git dpkg-dev make g++ gcc binutils libx11-dev libxpm-dev \
libxft-dev libxext-dev gfortran libssl-dev libpcre3-dev \
xlibmesa-glu-dev libglew1.5-dev libftgl-dev \
libmysqlclient-dev libfftw3-dev libcfitsio-dev \
graphviz-dev libavahi-compat-libdnssd-dev \
libldap2-dev python-dev libxml2-dev libkrb5-dev \
libgsl0-dev libqt4-dev r-base r-base-dev python-pip
```

Install ROOT 5 in some `PATH`
```bash
git clone http://root.cern.ch/git/root.git
cd root
git checkout v5-34-00-patches
./configure
make

```
Add to your `.bashrc`
```bash
source PATH/root/bin/thisroot.sh
```

# Development of module to calculate one specific point

In [3]:
%%bash
#git clone  --recursive git@gitlab.com:restrepo/VBF_IDM.git

In [2]:
%%bash
cd madgraph/
git checkout -b v2.3.3
./bin/mg5_aMC install.dat > /tmp/kk 2> /tmp/kkk
cd ..

Switched to a new branch 'v2.3.3'


In [3]:
LambdasL=[0.01,0.02,0.05,0.07,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,1.0,3.0,5.0,7.0,10.0]

In [4]:
def preamble(MHc=750,MH0=110):
    return '''import model InertDoublet_UFO
define p = g u c d s b u~ c~ d~ s~ b~
define j = p  
define l+ = e+ mu+ 
define l- = e- mu- 
define vl = ve vm vt 
define vl~ = ve~ vm~ vt~

generate p p > h2 h2 j j @0

output ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL

'''

def lamL_loop(MHc=750,MH0=110,lamL=0.01):
    return '''launch ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP'''+str(int(MHc))+'_'+str(int(MH0))+'''.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL '''+str(lamL)+'''

''' 


def closing(MHc=750,MH0=110):
    return '''launch ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL -i
print_results --path=./result_BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lamdaL_mh0_110.txt --format=short

done
'''

In [7]:
from subprocess import run

In [8]:
run('mkdir -p studies/IDM/Task_Asana').stderr

''

In [9]:
f=open('studies/IDM/Task_Asana/BP_750_A_110.txt','w')
f.write( preamble(MHc=750,MH0=110) )
for lamL in LambdasL:
    f.write( lamL_loop(MHc=750,MH0=110,lamL=lamL) )
f.write( closing(MHc=750,MH0=110) )
f.close()

```bash
# pip3 install subprocess.run && #pip2 install subprocess.run 
```

In [4]:
import  subprocess

In [18]:
r=run('ls -l khfajo')

In [22]:
run?

## Trash area

Temporal area where preliminary approaches are tried for.